In [3]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load video
cap = cv2.VideoCapture("video1.mp4")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process outputs to detect players
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Filter weak detections
                # Get player coordinates
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                # Draw bounding box
                cv2.rectangle(frame, (center_x - w // 2, center_y - h // 2), 
                              (center_x + w // 2, center_y + h // 2), 
                              (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_io.cpp:991: error: (-213:The function/feature is not implemented) Transpose the weights (except for convolutional) is not implemented in function 'cv::dnn::darknet::ReadDarknetFromWeightsStream'


In [ ]:
def generate_heatmap(positions_over_time, frame_shape):
    heatmap = np.zeros(frame_shape[:2])

    for positions in positions_over_time:
        for (x, y) in positions:
            heatmap[y, x] += 1

    return heatmap

# Generate heatmap from player positions
heatmap = generate_heatmap(positions_over_time, frame.shape)

# Plot heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap, cmap='viridis')
plt.title('Player Movement Heatmap')
plt.show()


TypeError: 'NoneType' object is not subscriptable

In [ ]:
def plot_player_positions(positions_over_time):
    all_positions = np.concatenate(positions_over_time, axis=0)
    x_coords, y_coords = zip(*all_positions)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(x_coords, y_coords, c='blue', alpha=0.5, s=10)
    plt.title('Player Movement Scatter Plot')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.show()

# Plot player positions over the entire video
plot_player_positions(positions_over_time)


In [ ]:
from sklearn.cluster import KMeans

# Reshape positions for clustering
all_positions = np.concatenate(positions_over_time, axis=0)
positions_array = np.array(all_positions)

# Apply KMeans clustering to identify movement patterns
kmeans = KMeans(n_clusters=3)
kmeans.fit(positions_array)

# Visualize the clusters
plt.figure(figsize=(10, 6))
plt.scatter(positions_array[:, 0], positions_array[:, 1], c=kmeans.labels_, cmap='rainbow', s=10)
plt.title('Player Movement Clustering')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.show()
